In [ ]:
# Take in a pdf or a word document and generate a word cloud of relavant skills and then print it out
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

In [ ]:
!pip install PyMuPDF
!pip install joblib
!pip install pymupdf
!pip install PyPDF2

In [ ]:
# !pip install frontend

In [ ]:
# Import Necessary Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import nltk
import string
import joblib
import re
import fitz
import joblib

from sklearn.naive_bayes import MultinomialNB
from sklearn.multiclass import OneVsRestClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import KNeighborsClassifier
from nltk.corpus import stopwords
from scipy.sparse import hstack
from google.colab import files

In [ ]:
# Resume Dataset Reference: https://www.kaggle.com/datasets/gauravduttakiit/resume-dataset
data1= pd.read_csv('/content/drive/MyDrive/UpdatedResumeDataSet.csv',encoding='utf-8')
data2= pd.read_csv('/content/drive/MyDrive/updated_resume_data.csv',encoding='utf-8')

category_column_1 = data1['Category']
resume_column_1 = data1['Resume']
category_column_2 = data2['Category']
resume_column_2 = data2['Resume']

combined_category = pd.concat([category_column_1, category_column_2], ignore_index=True)
combined_resume = pd.concat([resume_column_1, resume_column_2], ignore_index=True)
data = pd.DataFrame({'Category': combined_category, 'Resume': combined_resume})

data['Final_Resume'] = ''

data.head()


In [ ]:
data.info()

print("Displaying the distinct categories of resume:\n\n ")
print(data['Category'].unique())

print("Displaying the distinct categories of resume and the number of records belonging to each category:\n\n")
print(data['Category'].value_counts())

In [ ]:
def cleanResume(resumeText):
    resumeText = re.sub('http\S+\s*', ' ', resumeText)
    resumeText = re.sub('RT|cc', ' ', resumeText)
    resumeText = re.sub('#\S+', '', resumeText)
    resumeText = re.sub('@\S+', '  ', resumeText)
    resumeText = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', resumeText)
    resumeText = re.sub(r'[^\x00-\x7f]',r' ', resumeText)
    resumeText = re.sub('\s+', ' ', resumeText)
    return resumeText

data['Final_Resume'] = data.Resume.apply(lambda x: cleanResume(x))
data.head()

nltk.download('stopwords')
nltk.download('punkt')

oneSetOfStopWords = set(stopwords.words('english')+['``',"''"])

totalWords =[]
Sentences = data['Resume'].values
cleanedSentences = ""

for records in Sentences:
    cleanedText = cleanResume(records)
    cleanedSentences += cleanedText
    requiredWords = nltk.word_tokenize(cleanedText)
    for word in requiredWords:
        if word not in oneSetOfStopWords and word not in string.punctuation:
            totalWords.append(word)

wordfreqdist = nltk.FreqDist(totalWords)
mostcommon = wordfreqdist.most_common(50)

var_mod = ['Category']
le = LabelEncoder()
for i in var_mod:
  data[i] = le.fit_transform(data[i])
data.head()

In [ ]:
Text = data['Final_Resume'].values
Target = data['Category'].values

word_vectorizer = TfidfVectorizer(sublinear_tf=True,stop_words='english')

word_vectorizer.fit(Text)

WordFeatures = word_vectorizer.transform(Text)

X_train, X_test, Y_train, Y_test = train_test_split(WordFeatures, Target, random_state=42, test_size=0.2, shuffle=True, stratify=Target)

print(X_train.shape)
print(X_test.shape)

clf = OneVsRestClassifier(KNeighborsClassifier())

# changes
# clf.fit(X_train, Y_train)
# prediction = clf.predict(X_test)
# changes

# new code
from sklearn.metrics import accuracy_score

# Fit the classifier
clf.fit(X_train, Y_train)

# Predict on training and test set
train_pred = clf.predict(X_train)
test_pred = clf.predict(X_test)

# Calculate accuracy
train_accuracy = accuracy_score(Y_train, train_pred)
test_accuracy = accuracy_score(Y_test, test_pred)

# Plot the training and test accuracy curve
plt.plot([1, 2], [train_accuracy, test_accuracy], marker='o')
plt.title('Training and Test Accuracy')
plt.xticks([1, 2], ['Training', 'Test'])
plt.ylabel('Accuracy')
plt.show()
# new code

# changes
# print('Accuracy on training set: {:.2f}'.format(clf.score(X_train, Y_train)))
# print('Accuracy on test set:     {:.2f}'.format(clf.score(X_test, Y_test)))
# changes

In [ ]:
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neighbors import KNeighborsClassifier

# Assuming Text, Target, and data are defined

Text = data['Final_Resume'].values
Target = data['Category'].values

word_vectorizer = TfidfVectorizer(sublinear_tf=True, stop_words='english')

word_vectorizer.fit(Text)

WordFeatures = word_vectorizer.transform(Text)

X_train, X_test, Y_train, Y_test = train_test_split(WordFeatures, Target, random_state=42, test_size=0.2, shuffle=True, stratify=Target)

print(X_train.shape)
print(X_test.shape)

clf = OneVsRestClassifier(KNeighborsClassifier())

# Fit the classifier
clf.fit(X_train, Y_train)

# Predict on training and test set
train_pred = clf.predict(X_train)
test_pred = clf.predict(X_test)

# Calculate accuracy
train_accuracy = accuracy_score(Y_train, train_pred)
test_accuracy = accuracy_score(Y_test, test_pred)

# Plot the training and test accuracy curve
plt.plot([1, 2], [train_accuracy, test_accuracy], marker='o')
plt.title('Training and Test Accuracy')
plt.xticks([1, 2], ['Training', 'Test'])
plt.ylabel('Accuracy')

# Calculate slope for training curve
train_slope = (test_accuracy - train_accuracy) / (2 - 1)
print("Slope of training curve:", train_slope)

plt.show()

print('Accuracy on training set: {:.2f}'.format(train_accuracy))
print('Accuracy on test set:     {:.2f}'.format(test_accuracy))


In [ ]:
joblib.dump(clf, 'clf_model_new.pkl')

In [ ]:
clf = joblib.load('clf_model_new.pkl')

def preprocess_uploaded_pdf(uploaded_file):
    text = extract_text_from_pdf(uploaded_file)

    cleaned_resume = cleanResume(text)

    return cleaned_resume

def extract_text_from_pdf(pdf_content):
    pdf_document = fitz.open("pdf", pdf_content)
    text = ""

    for page_number in range(pdf_document.page_count):
        page = pdf_document[page_number]
        text += page.get_text()

    pdf_document.close()
    return text

def predict_category_and_feature_vector(model, vectorizer, uploaded_resume):
    resume_features = vectorizer.transform([uploaded_resume])
    predicted_category = model.predict(resume_features)[0]
    confidence_score = model.predict_proba(resume_features).max()
    feature_vector = resume_features.toarray()[0]

    return predicted_category, confidence_score, feature_vector

uploaded = files.upload()
uploaded_pdf_file = list(uploaded.values())[0]

cleaned_uploaded_resume = preprocess_uploaded_pdf(uploaded_pdf_file)

predicted_category1, confidence_score1, feature_vector1 = predict_category_and_feature_vector(clf, word_vectorizer, cleaned_uploaded_resume)

uploaded = files.upload()


uploaded_pdf_file = list(uploaded.values())[0]

cleaned_uploaded_resume = preprocess_uploaded_pdf(uploaded_pdf_file)

predicted_category2, confidence_score2, feature_vector2 = predict_category_and_feature_vector(clf, word_vectorizer, cleaned_uploaded_resume)

print(f"Predicted Category: {predicted_category1}")
print(f"Confidence Score: {confidence_score1:.4f}")
print("Feature Vector:")
print(feature_vector1)

print(f"Predicted Category: {predicted_category2}")
print(f"Confidence Score: {confidence_score2:.4f}")
print("Feature Vector:")
print(feature_vector2)


In [ ]:
def preprocess_uploaded_pdf(uploaded_file):
    text = extract_text_from_pdf(uploaded_file)
    cleaned_resume = cleanResume(text)

    return cleaned_resume

def extract_text_from_pdf(pdf_content):
    pdf_document = fitz.open("pdf", pdf_content)
    text = ""

    for page_number in range(pdf_document.page_count):
        page = pdf_document[page_number]
        text += page.get_text()

    pdf_document.close()
    return text

def predict_category_and_feature_vector(model, vectorizer, uploaded_resume):
    resume_features = vectorizer.transform([uploaded_resume])
    predicted_category = model.predict(resume_features)[0]
    confidence_score = model.predict_proba(resume_features).max()
    feature_vector = resume_features.toarray()[0]

    return predicted_category, confidence_score, feature_vector

uploaded = files.upload()

uploaded_pdf_file = list(uploaded.values())[0]

cleaned_uploaded_resume = preprocess_uploaded_pdf(uploaded_pdf_file)

predicted_category, confidence_score, feature_vector = predict_category_and_feature_vector(clf, word_vectorizer, cleaned_uploaded_resume)

print(f"Predicted Category: {predicted_category}")
print(f"Confidence Score: {confidence_score:.4f}")
print("Feature Vector:")
print(feature_vector)


In [ ]:
for value in feature_vector:
  print(value)

In [ ]:
all_feature_vectors = [feature_vector1, feature_vector2]  # Add more feature vectors as needed

target_feature_vector = feature_vector

cosine_similarities = cosine_similarity(target_feature_vector.reshape(1, -1), all_feature_vectors).flatten()

ranking = list(enumerate(cosine_similarities))

ranking.sort(key=lambda x: x[1], reverse=True)

print("Ranking of Resumes:")
for rank, (resume_index, similarity_score) in enumerate(ranking, 1):
    print(f"Rank {rank}: Resume {resume_index + 1} - Similarity Score: {similarity_score:.4f}")


In [ ]:
print("Target Feature Vector:")
print(target_feature_vector)

print("All Feature Vectors:")
for idx, fv in enumerate(all_feature_vectors, 1):
    print(f"Resume {idx} Feature Vector: {fv}")


In [ ]:
category_mapping = dict(zip(range(len(le.classes_)), le.classes_))
print(category_mapping)


In [ ]:
print(feature_vector.shape)